In [ ]:
!pip install -q torchvision pandas scikit-learn matplotlib

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("socal2.csv")
df.head()

In [ ]:
df = df[df['file'].notnull()]
df = df[df['file'].str.endswith('.jpg')]
tabular_features = ['bed', 'bath', 'acre_lot', 'house_size']
target_column = 'price'
image_column = 'file'
df = df.dropna(subset=tabular_features + [target_column, image_column])

In [ ]:
class HousePriceDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform
        self.tabular_cols = ['bed', 'bath', 'acre_lot', 'house_size']
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        image_path = os.path.join(self.image_dir, row['file'])
        image = read_image(image_path).float() / 255.0
        if self.transform:
            image = self.transform(image)
        tabular = torch.tensor(row[self.tabular_cols].values, dtype=torch.float32)
        price = torch.tensor(row['price'], dtype=torch.float32)
        return image, tabular, price

In [ ]:
resnet = resnet18(pretrained=True)
resnet.fc = torch.nn.Identity()
resnet.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224))
])

In [ ]:
dataset = HousePriceDataset(df, image_dir='Images', transform=transform)
loader = DataLoader(dataset, batch_size=16, shuffle=False)

image_features = []
tabular_data = []
prices = []

with torch.no_grad():
    for imgs, tabs, ys in tqdm(loader):
        feats = resnet(imgs)
        image_features.append(feats.numpy())
        tabular_data.append(tabs.numpy())
        prices.append(ys.numpy())

X_img = np.vstack(image_features)
X_tab = np.vstack(tabular_data)
y = np.concatenate(prices)

X_combined = np.concatenate([X_img, X_tab], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f"✅ MAE: {mae:.2f}")
print(f"✅ RMSE: {rmse:.2f}")

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual vs Predicted House Prices")
plt.show()